Python Implementation of Vector Symbolic Architectures:

In [2]:
# Mounted Drive to load GardensPointWalking image set for testing
# Image set is provided in GitHub Repo

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**seq_slam function:**

seqSLAMConv function is an integral part of the SEQUENCE-SLAM algorithm, which is used for identifying similar sequences in a database of images or patterns.

It takes a difference matrix—where each element represents the dissimilarity between two images—and a sequence length as inputs. The function then applies a series of convolutions across varying velocity scales to this difference matrix.

These convolutions are designed to enhance the similarity of sequential image matches over isolated matches. The velocity scale adjusts the convolution filter to account for varying motion speeds in the sequences.

After applying the convolutions, the function normalizes the results and combines them to produce a new difference matrix that emphasizes the strongest sequence matches.

In [5]:
import numpy as np
from scipy.ndimage import convolve

def seqSLAMConv(DD, n):
    v = np.arange(0.5, 1.6, 0.1)
    T = np.zeros((DD.shape[0], DD.shape[1], len(v)))
    supVal = 0
    prevFilter = None
    idx = 0
    for i in range(len(v)):
        H = getFilter(n, v[i])
        if not isSame(H, prevFilter):
            T[:,:,idx] = convolve(DD, H, mode='constant', cval=supVal)
            W = convolve(np.ones_like(DD), H, mode='constant')
            T[:,:,idx] /= W
            idx += 1
            prevFilter = H
    T[:,:,idx:] = []
    DDD = np.max(T, axis=2)
    return DDD

def getFilter(n, v):
    assert v > 0
    assert v >= 0.5
    nh = n // 2
    x = np.arange(1, nh+1)
    y = np.round(x * v).astype(int)
    hh = np.max(y)
    wh = np.max(x)
    idx = np.ravel_multi_index((y-1, x-1), (hh, wh))
    Hh = np.zeros((hh, wh))
    Hh.flat[idx] = 1
    H = np.block([[Hh, np.zeros((hh, wh+1))],
                  [np.zeros((1, wh)), 1, np.zeros((1, wh))],
                  [np.zeros((hh, wh+1)), Hh]])
    return H

def isSame(X, Y):
    return np.array_equal(X, Y)

**get_slsbh function:**


The get_sLSBH function is designed to generate Sparse Localized Spectral Binary Hashes (sLSBH) for a given dataset Y.

The process begins by determining a threshold n based on a sparsity parameter s, which dictates the number of significant components to retain. The function first performs a random projection of the data and then sorts it to identify the top n components, thereby sparsifying the data.

This operation is performed twice—once in descending order to create a hash L1 and once in ascending order to create L2. These two hashes are then concatenated to form a more robust binary hash L, which captures the essential features of the dataset while reducing its dimensionality.

This binary hash can be used in various tasks such as indexing, retrieval, and classification, providing a compressed yet informative representation of the data.

In [6]:
import numpy as np

def get_sLSBH(Y, s):
    n = round(Y.shape[1] * s)
    # random projection
    Y2 = Y.copy()
    # sort
    IDX = np.argsort(Y2, axis=1)[:, ::-1]
    # sparsification
    L1 = np.zeros(Y2.shape, dtype=np.float32)
    for i in range(Y2.shape[0]):
        L1[i, IDX[i, :n]] = 1
    # sort
    IDX = np.argsort(Y2, axis=1)
    # sparsification
    L2 = np.zeros(Y2.shape, dtype=np.float32)
    for i in range(Y2.shape[0]):
        L2[i, IDX[i, :n]] = 1
    # concat
    L = np.concatenate((L1, L2), axis=1)
    return L

**creatPR function:**


The createPR function generates precision-recall (PR) curves for evaluating sequence matching algorithms. Given a similarity matrix S, hard ground truth GThard, and optionally a soft ground truth GTsoft, it calculates the precision and recall for different threshold values.

If the removeDiagFlag is set, diagonal elements representing self-matches are removed from consideration. The singleMatchFlag ensures only the highest similarity scores are considered for each sequence, effectively enforcing a one-to-one match.

The function iteratively calculates precision and recall across the range of scores in S, identifying the threshold that yields the best F1 score. Additionally, the visualizePRAtThresh helper function creates a visual representation of true positives, false negatives, and false positives at the optimal threshold, aiding in the interpretability of the results.

In [7]:
import numpy as np
from scipy.sparse import csr_matrix

def createPR(S, GThard, GTsoft, removeDiagFlag, singleMatchFlag, evalAllSValuesFlag):
    if not evalAllSValuesFlag:
        evalAllSValuesFlag = 0
    GT = np.array(GThard, dtype=bool)

    if removeDiagFlag:
        if GTsoft.size > 0:
            seedIm = np.zeros_like(GT)
            seedIm[0, 0] = 1
            maindiaMask = np.logical_or(GTsoft, GTsoft.T)
            maindiaMask = np.logical_or(maindiaMask, np.eye(GT.shape[0], dtype=bool))
        else:
            maindiaMask = np.eye(GT.shape[0], dtype=bool)

        maindiaMask = np.logical_or(maindiaMask, np.tril(np.ones_like(S, dtype=bool)))
        S[maindiaMask] = np.min(S)
        GT[maindiaMask] = False

    if GTsoft.size > 0:
        S[np.logical_and(GTsoft, np.logical_not(GThard))] = np.min(S)

    if singleMatchFlag:
        hIdx = np.argmax(S, axis=0)
        hIdxInd = np.ravel_multi_index((hIdx, np.arange(S.shape[1])), S.shape)
        T = np.full_like(S, np.min(S))
        T.flat[hIdxInd] = S.flat[hIdxInd]
        S = T

    R = [0]
    P = [1]
    startV = np.max(S)
    endV = np.min(S)
    bestF = 0
    bestT = startV
    bestP = 0
    bestR = 0

    if evalAllSValuesFlag:
        s_vals = np.sort(np.unique(S))[::-1]
    else:
        s_vals = np.linspace(startV, endV, 100)

    GT_sparse = csr_matrix(GT)

    for i in s_vals:
        B = S >= i
        TP = np.count_nonzero(GT_sparse.multiply(B))
        FN = np.count_nonzero(GT_sparse.multiply(np.logical_not(B)))
        FP = np.count_nonzero(np.logical_not(GT) & B)
        P.append(TP / (TP + FP))
        R.append(TP / (TP + FN))
        F = 2 * P[-1] * R[-1] / (P[-1] + R[-1])
        if F > bestF:
            bestF = F
            bestT = i
            bestP = P[-1]
            bestR = R[-1]

    R.append(1)
    P.append(0)
    V = visualizePRAtThresh(S, GT, bestT)

    return P, R, V, bestP, bestR, bestF

def visualizePRAtThresh(S, GT, t):
    B = S >= t
    TP = GT & B
    FN = GT & np.logical_not(B)
    FP = np.logical_not(GT) & B
    V = np.dstack((FP, TP, FN))

    return V

**unbind_vectors operation:**

The unbind_vectors function is designed to reverse the binding operation between pairs of vectors in various Vector Symbolic Architecture (VSA) models. It takes as inputs two sets of vectors and the VSA model identifier, then performs model-specific unbinding operations. For instance, in MAP models, it executes element-wise multiplication, while for BSC, it uses an efficient XOR operation. The BSDC model requires a similarity search instead of direct unbinding. For HRR and FHRR, the function applies Fourier transform-based involution and complex arithmetic, respectively. In the BSDC_SHIFT model, it involves array shifting, and for BSDC_SEG, it handles sparse segments. The function showcases the diverse nature of unbinding across different VSA models, reflecting the need for tailored approaches to effectively reverse the binding of high-dimensional vectors.

In [8]:
import numpy as np

def unbind_vectors(vsa, vectors_1, vectors_2, **kwargs):
    # scken, 2020
    default_density = 1/np.sqrt(vectors_1.shape[0])  # density computing is optained from rachkovskji (most capacity and good stability)

    density = kwargs.get('density', default_density)

    if vsa in ['MAP_B', 'MAP_C', 'MAP_I']:
        # elementwise multiplication
        unbound_vectors = vectors_1 * vectors_2
    elif vsa == 'BSC':
        # efficient xor implementation for multi inputs
        unbound_vectors = np.double(np.add(vectors_1, vectors_2) == 1)
    elif vsa == 'BSDC':
        # find the most similar item in item_mem
        print('There is no specific unbind operator for the selected VSA - use the finding of the most similar vectors in item memory instead!')
    elif vsa == 'BSDC_SHIFT':
        # calculate the shift number (sum of all ones-index)
        idx = np.arange(1, vectors_1.shape[0]+1).dot(vectors_1)
        # shift each column with specific index number
        unbound_vectors = np.zeros(vectors_1.shape)
        for i in range(idx.size):
            unbound_vectors[:, i] = np.roll(vectors_2[:, i], -idx[i])
    elif vsa == 'HRR':
        # involution as approximate inverse
        inverse = np.vstack((vectors_1[0, :], np.flip(vectors_1[1:, :], axis=0)))
        unbound_vectors = np.fft.ifft(np.fft.fft(inverse, vectors_1.shape[0], axis=0) * np.fft.fft(vectors_2, vectors_2.shape[0], axis=0), axis=0)
    elif vsa == 'FHRR':
        # complex multiplication with negative 'role' vector
        unbound_vectors = np.arctan2(np.sin(vectors_2 - vectors_1), np.cos(vectors_2 - vectors_1))
    elif vsa == 'BSDC_SEG':
        # sparse vectors with segements
        dim = vectors_1.shape[0]
        num_segments = int(np.floor(dim * density))
        num_vecs = vectors_1.shape[1]
        size_segments = int(np.floor(dim / num_segments))
        role = vectors_1[:num_segments*size_segments, :]
        filler = vectors_2[:num_segments*size_segments, :]
        # first part of the vector
        role_segments = np.reshape(role, (size_segments, num_segments, num_vecs))
        filler_segments = np.reshape(filler, (size_segments, num_segments, num_vecs))
        role_idx = np.where(role_segments)
        filler_idx = np.where(filler_segments)
        role_rows, role_cols, role_tables = np.unravel_index(role_idx, role_segments.shape)
        filler_rows, filler_cols, filler_tables = np.unravel_index(filler_idx, filler_segments.shape)
        result_rows = np.mod(filler_rows - role_rows[filler_cols*filler_tables] - 1, size_segments) + 1
        unbound_vectors = np.zeros(role_segments.shape)
        idx = np.ravel_multi_index((result_rows, filler_cols, filler_tables), role_segments.shape)
        unbound_vectors.flat[idx] = 1
        unbound_vectors = np.reshape(unbound_vectors, (size_segments*num_segments, num_vecs))
        # if there is a remain part
        unbound_vectors_part2 = np.empty((0, num_vecs))
        if num_segments*size_segments != dim:
            filler = vectors_2[num_segments*size_segments:, :]
            unbound_vectors_part2 = filler
        unbound_vectors = np.vstack((unbound_vectors, unbound_vectors_part2))
    else:
        print('Representation is not defined!')

    return unbound_vectors

**get_ngram operation:**


The getNgram function constructs n-grams using a specified Vector Symbolic Architecture (VSA) model. An n-gram is a contiguous sequence of n items from a given sequence of text or speech. This function translates a sequence of keys into their corresponding character codes, then iteratively binds and permutes character vectors from an item memory to form the n-gram representation. It demonstrates how sequences, such as words or phrases, can be encoded into the high-dimensional vector space characteristic of VSAs, preserving the sequential nature of the input through the VSA's binding and permutation operations.

In [9]:
def getNgram(VSA, keys, char_item_memory, seq_item_memory):
    key_codes = [ord(key) for key in keys]
    num_ngrams = len(key_codes)
    # create ngrams
    ngram = VSA.permute(char_item_memory[:, key_codes[0]])
    for i in range(1, num_ngrams):
        ngram = VSA.bind(ngram, VSA.permute(char_item_memory[:, key_codes[i]], i))
    return ngram

**generate_vectors operation:**


The generate_vectors function is a versatile tool for generating high-dimensional vectors according to different Vector Symbolic Architectures (VSAs). Depending on the VSA specified, it can produce dense or sparse binary vectors, continuous uniform or normal distributions, or even complex-valued vectors.

This function allows for the customization of the vector dimensionality, density of non-zero elements in sparse representations, and the number of vectors to generate. It's particularly useful for creating the foundational vector space for various VSA operations, such as binding, bundling, and unbinding, which are essential in cognitive computing tasks. Whether for binary sparse coding, holographic reduced representations, or Fourier holographic representations, this function provides the necessary building blocks for VSA-based computations.

In [10]:
import numpy as np

def generate_vectors(dim=10000, vsa='map', num=1, density=-1):
    default_dim = 10000
    default_vsa = 'map'
    default_density = -1
    default_num = 1

    if dim is None:
        dim = default_dim
    if vsa is None:
        vsa = default_vsa
    if num is None:
        num = default_num
    if density is None:
        density = default_density

    if density == -1:
        if vsa in ['BSDC', 'BSDC_test', 'BSDC_SHIFT', 'BSDC_THIN']:
            density = 1/np.sqrt(dim)
        elif vsa == 'BSDC_25':
            density = 0.25
        elif vsa == 'BSDC_SEG':
            density = 1/np.sqrt(dim)
        else:
            density = 0.5

    if vsa in ['MAP_B', 'MAP_I']:
        gen_vecs = np.random.choice([-1, 1], size=(dim, num))
    elif vsa == 'MAP_C':
        gen_vecs = np.random.uniform(low=-1, high=1, size=(dim, num))
    elif vsa == 'BSC':
        gen_vecs = np.random.choice([0, 1], size=(dim, num))
    elif vsa in ['BSDC', 'BSDC_SHIFT', 'BSDC_25', 'BSDC_THIN']:
        rand_values = np.random.uniform(low=0, high=1, size=(dim, num))
        rows = np.argpartition(rand_values, -int(dim*density), axis=0)[-int(dim*density):]
        values = np.zeros_like(rand_values)
        values[rows, np.arange(num)] = 1
        gen_vecs = values
    elif vsa == 'BSDC_SEG':
        num_segments = int(dim*density)
        size_segments = int(dim/num_segments)
        z = np.zeros((size_segments, num_segments, num))
        rand_r = np.random.randint(low=0, high=size_segments, size=(num_segments, num))
        rand_c = np.repeat(np.arange(num_segments)[:, np.newaxis], num, axis=1)
        rand_t = np.repeat(np.arange(num)[np.newaxis, :], num_segments, axis=0)
        indices = np.stack((rand_r, rand_c, rand_t), axis=-1)
        z[tuple(indices.T)] = 1
        gen_vecs = z.reshape(-1, num)
        if dim != num_segments*size_segments:
            d = dim - num_segments*size_segments
            gen_vecs = np.concatenate((gen_vecs, np.zeros((d, num))), axis=0)
    elif vsa in ['HRR', 'HRR_VTB', 'MBAT']:
        gen_vecs = np.random.normal(loc=0, scale=np.sqrt(1/dim), size=(dim, num))
    elif vsa in ['FHRR', 'FHRR_fft', 'FHRR_cos']:
        gen_vecs = np.random.uniform(low=-np.pi, high=np.pi, size=(dim, num))
    else:
        print('Representation is not defined!')

    return gen_vecs

**fractional_bindings operation:**

The fractional_binding function implements fractional binding operations in various Vector Symbolic Architecture (VSA) models. Fractional binding is a technique to partially bind vectors together, allowing for more nuanced and flexible representations.

For models like FHRR (Fourier Holographic Reduced Representations), it performs modulo arithmetic on the vector elements, effectively scaling them by a factor k.

In BSDC-based models (Binary Sparse Distributed Codes) and BSC (Binary Spatter Codes), the function utilizes the Fast Fourier Transform (FFT) to raise the vectors to the power of k and then retrieves the binary representation.

This function demonstrates a key aspect of VSA - the ability to modify the traditional binding operation to capture varying degrees of association between vectors, which is vital for complex cognitive tasks.

In [11]:
import numpy as np

def fractional_binding(vsa, vector, k):
    if vsa == 'FHRR' or vsa == 'FHRR_fft':
        v = np.remainder(np.tile(vector, np.shape(k)) * k, np.pi)
    elif vsa == 'BSDC' or vsa == 'BSDC_SEG' or vsa == 'BSDC_SHIFT' or vsa == 'BSC' or vsa == 'BSDC_25':
        values = np.fft.ifft(np.fft.fft(np.tile(vector, (1, len(k))), vector.shape[0], 1) ** k, vector.shape[0], 1)
        v = np.angle(values) > 0
    else:
        values = np.fft.ifft(np.fft.fft(np.tile(vector, (1, len(k))), vector.shape[0], 1) ** k, vector.shape[0], 1)
        v = np.real(values)

    return v

**convert_vectors operation:**

The convert_vectors function is a key component in Vector Symbolic Architectures (VSAs) that adapts input vectors to suit various VSA models. This function takes an input matrix Y and a specified VSA type, then processes and converts the vectors to conform to the representation requirements of the chosen VSA.

For instance, in continuous VSAs like MAP_C, it clamps the values within a specific range, while in binary VSAs like BSC, it converts the values to binary form. The function also handles models like FHRR (Fourier Holographic Reduced Representations) by transforming vectors into their angular representation.

Additionally, for sparse binary models like BSDC, it performs sparsification based on a specified density parameter. This conversion step is crucial for ensuring that the input data is compatible with the specific operations and characteristics of the chosen VSA model.

In [12]:
import numpy as np
from scipy.stats import norm

def convert_vectors(vsa, Y, density=None):
    if density is None:
        # set default density
        if vsa in ['BSDC', 'BSDC_test', 'BSDC_SHIFT']:
            density = 1/np.sqrt(Y.shape[1])  # density computing is optains from rachkovskji (most capacity and good stability)
        elif vsa == 'BSDC_25':
            density = 0.25
        elif vsa == 'BSDC_SEG':
            density = 1/np.sqrt(Y.shape[1])
        else:
            density = 0.5

    if vsa == 'MAP_C':
        # convert
        values = Y
        values[values > 1] = 1
        values[values < -1] = -1
    elif vsa == 'map_trans_uniform':
        # convert
        values = Y
        for i in range(Y.shape[0]):
            pd = norm(loc=np.mean(Y[i,:]), scale=np.sqrt(np.var(Y[i,:])))
            values[i,:] = pd.cdf(Y[i,:]) * 2 - 1
    elif vsa in ['MAP_B', 'MAP_I']:
        # convert
        values = np.double(Y > 0) * 2 - 1
    elif vsa == 'BSC':
        # convert
        values = np.double(Y > 0)
    elif vsa in ['HRR', 'HRR_VTB', 'MBAT']:
        # convert
        values = np.linalg.norm(Y, axis=1)
    elif vsa == 'FHRR':
        # convert
        values = np.angle(np.fft.fft(Y, Y.shape[1], axis=1))
    elif vsa in ['BSDC', 'BSDC_SHIFT', 'BSDC_SEG']:
        # project values
        values = get_sLSBH(Y, density)
    elif vsa in ['NONE', 'Proj.']:
        # use the original vectors without converting
        values = Y
    else:
        print('Representation is not defined!')

    return values

**compute_sim operation:**

The compute_sim function is a versatile utility for calculating similarity between two sets of vectors using various similarity metrics, depending on the specified VSA (Vector Symbolic Architecture) type. It takes two sets of vectors, vectors_1 and vectors_2, and computes a similarity matrix between them.

The similarity calculation method varies based on the chosen VSA type:

- Cosine Similarity (MAP_B, MAP_C, HRR, HRR_VTB, NONE, MAP_I, MBAT, Proj,

- FHRR_cos): For continuous VSAs (MAP_C, HRR), it calculates the cosine similarity between vectors. For binary VSAs (MAP_B, MAP_I), it uses a modified cosine similarity. For complex-valued VSAs (MBAT, FHRR_cos), it computes the average cosine of the angle between vectors.

- Hamming Distance (BSC): For binary sparse coding (BSC), it computes the Hamming distance between binary vectors.

- Overlap (BSDC, BSDC_SHIFT, BSDC_25, BSDC_SEG, BSDC_THIN): For various forms of binary sparse distributed representations (BSDC), it calculates the overlap between binary vectors.
Average Cosine of Distance (FHRR_fft, FHRR):

- For Fourier-based Holographic Reduced Representations (FHRR), it computes the average cosine of the distance between complex-valued vectors.

The function first transposes the input arrays to ensure proper alignment for the similarity calculations. It then initializes a similarity matrix and proceeds to calculate the similarity values based on the selected VSA type. The choice of VSA determines which mathematical operation is performed to measure similarity. This function is essential for assessing the similarity between vectors, which is a fundamental operation in many VSA applications.

In [13]:
def compute_sim(vsa, vectors_1, vectors_2):
    # first transpose the arrays
    vectors_1 = vectors_1.T
    vectors_2 = vectors_2.T
    sim_matrix = np.zeros((vectors_1.shape[0], vectors_2.shape[0]))
    np.warnings.filterwarnings('ignore')
    if vsa in ['MAP_B', 'MAP_C', 'HRR', 'HRR_VTB', 'NONE', 'MAP_I', 'MBAT', 'Proj', 'FHRR_cos']:
        # cosine similarity
        sim_matrix = (vectors_1 / np.sqrt(np.sum(vectors_1**2, axis=1, keepdims=True))) @ (vectors_2 / np.sqrt(np.sum(vectors_2**2, axis=1, keepdims=True))).T
    elif vsa == 'BSC':
        # hamming distance
        sim_matrix = 1 - pdist2(vectors_1, vectors_2, metric='hamming')
    elif vsa in ['BSDC', 'BSDC_SHIFT', 'BSDC_25', 'BSDC_SEG', 'BSDC_THIN']:
        # overlap
        vectors_1 = vectors_1 > 0
        vectors_2 = vectors_2 > 0
        sim_matrix = vectors_1 @ vectors_2.T
    elif vsa in ['FHRR_fft', 'FHRR']:
        # average of cosine of distance
        # convert to complex values
        sim_matrix = np.real(np.exp(vectors_1*1j) @ np.exp(vectors_2*1j).T) / vectors_1.shape[1]
    else:
        print('Representation is not defined!')
    np.warnings.filterwarnings('default')
    return sim_matrix

**complex_sim operation:**


The complexSim function calculates the similarity between two sets of complex-valued vectors, typically used in the context of Fourier-based Holographic Reduced Representations (FHRR). It computes the similarity by measuring the cosine of the angle between pairs of complex numbers in the vectors. The resulting similarity matrix indicates how similar the corresponding pairs of vectors are, with higher values indicating greater similarity.

In [14]:
import numpy as np

def complexSim(v1, v2):
    # compute the similarity between vectors of angles (from complex numbers --> FHRR)
    sim_matrix = np.zeros((len(v1), len(v2)))
    for i in range(len(v1)):
        sim_matrix[i] = np.sum(np.cos(v1[i] - v2), axis=1) / len(v1[i])
    return sim_matrix

**cdt operation:**

The cdt function performs a Convolutive Displacement Transformation (CDT) operation on a superposition vector. This operation aims to reduce the density of activated elements in the superposition vector. It iteratively shifts the vector, performs element-wise logical AND with the original vector, and sets elements to zero based on certain criteria. This process continues until the mean density of activated elements across columns falls below a specified threshold (density) or until a maximum number of iterations (max_iters) is reached. The result is a modified superposition vector with reduced density, which can be useful for various operations in vector symbolic architectures.

In [15]:
def cdt(superposition_vector, max_iters, density):
    Z = superposition_vector
    counter = 1
    while np.mean(np.sum(Z, axis=0) / Z.shape[0]) > density:
        r = counter  # determine the shifting
        r = counter  # determine the shifting
        permutation = np.roll(superposition_vector, r)
        thinned = np.logical_and(superposition_vector, permutation)
        Z[thinned] = 0

        if counter > max_iters:  # if more than max_iters iterations, break loop
            break

        counter += 1

    return Z

**bundle_vectors operation:**

The bundle_vectors function combines two sets of vectors (vectors_1 and vectors_2) into a bundled vector representation based on a specified Vector Symbolic Architecture (VSA) method (vsa). It offers various methods for bundling vectors:

- For methods like 'MAP_B', 'MAP_C', 'MAP_I', and 'BSC', it combines vectors based on majority rules, sum, or hamming distance, with optional normalization and density control.

- 'BSDC' performs element-wise disjunction with optional normalization and density control.

- 'HRR', 'HRR_VTB', and 'MBAT' methods add vectors element-wise, optionally normalizing them.

- 'FHRR' computes the average angle between vectors.

- 'BSDC_SHIFT' performs element-wise disjunction with a density-based selection of elements.

- 'BSDC_SEG' bundles vectors in segments with optional normalization and density control.

The resulting bundled vector representation is returned, which can be used in various vector symbolic operations.

In [16]:
import numpy as np

def bundle_vectors(vsa, vectors_1, vectors_2, normalize=True, density=0.5, max_density=1):
    # concatenate the two input vectors
    vector_array = np.concatenate((vectors_1, vectors_2), axis=1)
    dim = vector_array.shape[0]
    n_dim = vectors_1.ndim

    if vsa == 'MAP_B':
        # majority rule
        values = np.sum(vector_array, axis=n_dim)
        if normalize:
            values[values < -1] = -1
            values[values > 1] = 1
            random_choice = np.random.choice([-1, 1], size=(dim, 1))
            values[values == 0] = random_choice[values == 0]
        bundled_vectors = values
    elif vsa == 'MAP_C':
        values = np.sum(vector_array, axis=n_dim)
        if normalize:
            # normalization of bundled vectors
            values[values > 1] = 1
            values[values < -1] = -1
        bundled_vectors = values
    elif vsa == 'MAP_I':
        # sum
        bundled_vectors = np.sum(vector_array, axis=n_dim)
    elif vsa == 'BSC':
        values = np.sum(vector_array, axis=n_dim)
        if normalize:
            # check if number of vectors is odd (apply majority rule)
            number_vec = vector_array.shape[1]
            if number_vec % 2 == 0:
                random_choice = np.random.choice([0, 1], size=(dim, 1))
                values = values + random_choice
                number_vec = number_vec + 1
            thresh = number_vec / 2
            # if threshold highly differ from mean, than use mean
            # as threshold
            if abs(thresh - np.mean(values)) > 2:
                thresh = np.mean(values)
            values = np.array(values > thresh, dtype=int)
        bundled_vectors = values
    elif vsa == 'BSDC':
        # elementwise disjunction
        # if normalize true, thinning of the resulting bundle
        k = int(max_density * vector_array.shape[0])
        values = np.sum(vector_array, axis=n_dim)
        if normalize:
            bundled_vectors = np.zeros((vector_array.shape[0], 1))
            idx = np.argpartition(values, -k)[-k:]
            bundled_vectors[idx] = values[idx] > 0
        else:
            bundled_vectors = np.array(values >= 1, dtype=int)
    elif vsa in ['HRR', 'HRR_VTB', 'MBAT']:
        # elementwise addition
        values = np.sum(vector_array, axis=n_dim)
        if normalize:
            values = values / np.linalg.norm(values)
        bundled_vectors = values
    elif vsa == 'FHRR':
        # average angle
        vectors = np.concatenate((vectors_1, vectors_2), axis=1)
        vcos = np.cos(vectors)
        vsin = np.sin(vectors)
        vcos_sum = np.sum(vcos, axis=n_dim)
        vsin_sum = np.sum(vsin, axis=n_dim)
        if normalize:
            values = np.arctan2(vsin_sum, vcos_sum)
        else:
            values = vcos + 1j * vsin
        bundled_vectors = values
    elif vsa == 'BSDC_SHIFT':
        # elementwise disjunction
        # select the k highest values (k is computed with the density)
        k = int(max_density * vector_array.shape[0])
        values = np.sum(vector_array, axis=n_dim)
        if normalize:
            bundled_vectors = np.zeros((vector_array.shape[0], 1))
            idx = np.argpartition(values, -k)[-k:]
            bundled_vectors[idx] = values[idx] > 0
        else:
            bundled_vectors = np.array(values > 0, dtype=int)
    elif vsa == 'BSDC_SEG':
        num_segments = int(dim * density)
        size_segments = int(dim / num_segments)
        k = int(max_density * size_segments)
        values = np.sum(vector_array, axis=n_dim)
        if normalize:
            values_segments = values[:size_segments * num_segments].reshape((size_segments, num_segments))
            idx = np.argpartition(values_segments, -k, axis=0)[-k:]
            idx = np.ravel_multi_index((idx, np.arange(num_segments)), values_segments.shape)
            bundled_vectors = np.zeros(values_segments.shape)
            bundled_vectors.ravel()[idx] = values_segments.ravel()[idx]
            bundled_vectors = np.ravel(bundled_vectors) > 0
        else:
            bundled_vectors = values
        if size_segments * num_segments != values.shape[0]:
            bundled_vectors = np.concatenate((bundled_vectors, values[size_segments * num_segments:]))
    else:
        print('Representation is not defined!')

    bundled_vectors = bundled_vectors.astype(float)
    return bundled_vectors

**bind_vectors operation:**

The bind_vectors function performs vector binding operations based on the specified Vector Symbolic Architecture (VSA) method (vsa). It combines two sets of vectors (vectors_1 and vectors_2) into a single bound vector representation. Here's a brief description of what it does for each method:

- For 'MAP_B', 'MAP_C', and 'MAP_I', it performs element-wise multiplication between the vectors.

- 'BSC' efficiently computes XOR operations between vectors.

- 'BSDC' combines vectors using the CDT (Conscious Dynamic Thinning) algorithm, which involves iterative thinning to control density.

- 'BSDC_SHIFT' calculates the shift number based on the dot product with a range of values and then shifts the columns of the second set of vectors accordingly.

- 'HRR' uses circular convolution to bind vectors.

- 'FHRR' computes the binding by taking the modulus of the sum of angles.

- 'BSDC_SEG' binds vectors in segments, where each segment is bound independently.

The resulting bound vector representation is returned for further use in vector symbolic operations.

In [17]:
import numpy as np

def bind_vectors(vsa, vectors_1, vectors_2, **kwargs):
    default_density = 1/np.sqrt(vectors_1.shape[0])
    density = kwargs.get('density', default_density)

    bound_vectors = None

    if vsa in ['MAP_B', 'MAP_C', 'MAP_I']:
        bound_vectors = vectors_1 * vectors_2
    elif vsa == 'BSC':
        bound_vectors = np.double(np.add(vectors_1, vectors_2) == 1)
    elif vsa == 'BSDC':
        values_disj = np.double(np.add(vectors_1, vectors_2))
        bound_vectors = cdt(values_disj, 50, 1/np.sqrt(vectors_1.shape[0]))
    elif vsa == 'BSDC_SHIFT':
        idx = np.arange(1, vectors_1.shape[0]+1).dot(vectors_1)
        bound_vectors = np.zeros(vectors_1.shape)
        for i in range(idx.size):
            bound_vectors[:, i] = np.roll(vectors_2[:, i], idx[i])
    elif vsa == 'HRR':
        ccirc = np.fft.ifft(np.fft.fft(vectors_1, vectors_1.shape[0], axis=0) * np.fft.fft(vectors_2, vectors_2.shape[0], axis=0), axis=0)
        bound_vectors = ccirc
    elif vsa == 'FHRR':
        bound_vectors = np.mod(vectors_1 + vectors_2 + np.pi, 2 * np.pi) - np.pi
    elif vsa == 'BSDC_SEG':
        dim = vectors_1.shape[0]
        num_segments = int(np.floor(dim * density))
        size_segments = int(np.floor(dim / num_segments))
        num_vecs = vectors_1.shape[1]

        role = vectors_1[:num_segments * size_segments, :]
        filler = vectors_2[:num_segments * size_segments, :]

        # Reshape vectors
        role_segments = np.reshape(role, (size_segments, num_segments, num_vecs), order='F')
        filler_segments = np.reshape(filler, (size_segments, num_segments, num_vecs), order='F')

        # Find indices
        role_idx = np.flatnonzero(role_segments)
        filler_idx = np.flatnonzero(filler_segments)

        # Convert indices to subscripts
        role_rows, _, _ = np.unravel_index(role_idx, role_segments.shape)
        filler_rows, filler_cols, filler_tables = np.unravel_index(filler_idx, filler_segments.shape)

        # Calculate result rows
        result_rows = np.mod(role_rows[filler_cols * filler_tables] + filler_rows - 1, size_segments) + 1

        # Initialize bound_vectors
        bound_vectors = np.zeros_like(role_segments)

        # Convert subscripts back to indices and set values in bound_vectors
        idx = np.ravel_multi_index((result_rows, filler_cols, filler_tables), role_segments.shape)
        bound_vectors.flat[idx] = 1

        # Reshape bound_vectors
        bound_vectors = np.reshape(bound_vectors, (size_segments * num_segments, num_vecs), order='F')
    else:
        print('Representation is not defined!')

    return bound_vectors

In [18]:
import random,string
def generate_random_string(size):
        return ''.join(random.choice(string.ascii_letters + string.digits))

**vsa_env operation:**

The vsa_env class is a Python class that provides various Vector Symbolic Architecture (VSA) operations, and it can be used to work with different VSA implementations. Here's an overview of its key methods and functionalities:

- __init__: Initializes the VSA environment with specified parameters such as VSA type (vsa), dimension (dim), density (density), and maximum density (max_density).

- add_vector: Adds vectors to the environment. It generates random vectors if vec is not provided. Vectors can be associated with names for easy retrieval.

- sim: Computes similarity between two sets of vectors using different similarity metrics based on the selected VSA type.

- bind: Binds two sets of vectors together using the specified VSA type.

- unbind: Unbinds two sets of vectors using the specified VSA type.

- bundle: Bundles two sets of vectors together, with options for normalization and density control.

- permute: Performs vector permutation by rolling the elements of a vector.

- find_k_nearest: Finds the k-nearest vectors in the environment based on similarity.

- find_by_name: Retrieves vectors by their associated names.

- frac_binding: Performs fractional binding on a vector.

- convert: Converts vectors using the selected VSA type.

- generate_vectors: Generates random vectors based on the specified VSA type and parameters.

- compute_sim: Computes similarity between two sets of vectors using different similarity metrics.

- bind_vectors: Binds two sets of vectors together using the specified VSA type and density.

- unbind_vectors: Unbinds two sets of vectors using the specified VSA type and density.

- bundle_vectors: Bundles two sets of vectors together, with options for normalization and density control.

- fractional_binding: Performs fractional binding on a vector.

- convert_vectors: Converts vectors using the selected VSA type and density.

This class supports various VSA implementations and perform operations like binding, unbinding, and similarity computations on sets of vectors.

In [19]:
# Python class of different VSA implementation
import numpy as np
from scipy.spatial.distance import hamming

class vsa_env:
    def __init__(self, vsa='MAP_B', dim=10000, density=-1, max_density=1):
        self.dim = dim
        self.vsa = vsa
        self.item_mem = [[], []]
        self.max_density = max_density
        if density == -1:
            if vsa in ['BSDC', 'BSDC_SHIFT']:
                density = 1 / np.sqrt(dim)
            elif vsa == 'BSDC_25':
                density = 0.25
            elif vsa == 'BSDC_SEG':
                density = 1 / np.sqrt(dim)
            else:
                density = 0.5
        self.density = density


    def add_vector(self, vec=None, name=-1, num=1, add_item=1, return_vector=1):
        if vec is None:
            vectors = self.generate_vectors(self.vsa, self.dim, num, self.density)
        else:
            vectors = vec
            self.item_mem[0] =  vectors
            num = vectors.shape[1]

        if add_item:
            self.item_mem[0] =  vectors

            if isinstance(name, int):
                rand_names = self.rnd_name(size=(8,num))
                self.item_mem[1]= rand_names
            else:
                names = self.item_mem[1]
                self.item_mem[1] = np.concatenate((names, [name]))
        if not return_vector:
            vectors = None

        return vectors

    def sim(self, vectors_1, vectors_2):
        return self.compute_sim(self.vsa, vectors_1, vectors_2)

    def bind(self, vectors_1, vectors_2):
        return self.bind_vectors(self.vsa, vectors_1, vectors_2, density=self.density)

    def unbind(self, vectors_1, vectors_2):
        return self.unbind_vectors(self.vsa, vectors_1, vectors_2, density=self.density)

    def bundle(self, vectors_1, vectors_2, normalize=None):
        if normalize is None:
            return self.bundle_vectors(self.vsa, vectors_1, vectors_2, density=self.density, max_density=self.max_density)
        else:
            return self.bundle_vectors(self.vsa, vectors_1, vectors_2, normalize=normalize, density=self.density, max_density=self.max_density)

    def permute(self, vector, p=1):
        return np.roll(vector, p)

    def find_k_nearest(self,vectors_in, k=1):
        sim_vec = self.sim(self.item_mem[0], vectors_in)
        idx = np.argsort(sim_vec)[::-1]
        sim_vec_sort = sim_vec[idx]
        s_highest = sim_vec_sort[:k]
        rows = idx[:k]
        names = self.item_mem[1]

        names = names[rows]
        #names = names.reshape((k, vectors_in.shape[1]))
        vecs = self.item_mem[0]
        vectors = vecs[:, rows]
        #vectors = vectors.reshape((vectors.shape[0], k, vectors_in.shape[1]))
        s = sim_vec[:k,:]
        s = s.reshape((k, vectors_in.shape[1]))

        return vecs, names, s

    def find_by_name(self, vector_name):
        idx = np.where(self.item_mem[1] == vector_name)[0]
        if len(idx) >= 1:
            vector = self.item_mem[0][:, idx]
        else:
            print('No vector for name {} found!'.format(vector_name))
            vector = None

        return vector

    def frac_binding(self, vector, k):
        return self.fractional_binding(self.vsa, vector, k)

    def convert(self, vector_array):
        return self.convert_vectors(self.vsa, vector_array, self.density)


    @staticmethod
    def rnd_name(size=(8,1)):
      # Convert the list of strings to a NumPy array
      strings = [generate_random_string(size[0]) for _ in range(size[1])]
      array_of_strings = np.array(strings, dtype=np.dtype('U8'))
      return array_of_strings

    @staticmethod
    def generate_vectors(vsa, dim, num, density):
        if vsa == 'MAP_C':
            vectors = np.random.choice([-1, 1], size=(dim, num))
        elif vsa == 'MAP_B':
            vectors = np.random.choice([0, 1], size=(dim, num), p=[1 - density, density])
        elif vsa == 'MAP_I':
            vectors = np.eye(dim)[:, :num]
        elif vsa == 'BSC':
            vectors = np.random.choice([-1, 1], size=(dim, num))
        elif vsa == 'BSDC':
            vectors = np.random.choice([-1, 1], size=(dim, num))
        elif vsa == 'BSDC_SHIFT':
            vectors = np.random.choice([-1, 1], size=(dim, num))
        elif vsa == 'HRR':
            vectors = np.random.choice([-1, 1], size=(dim, num))
        elif vsa == 'HRR_VTB':
            vectors = np.random.choice([-1, 1], size=(dim, num))
        elif vsa == 'FHRR':
            vectors = np.random.choice([-1, 1], size=(dim, num))
        elif vsa == 'BSDC_SEG':
            vectors = np.random.choice([-1, 1], size=(dim, num))
        elif vsa == 'MBAT':
            vectors = np.random.choice([-1, 1], size=(dim, num))
        else:
            vectors = np.random.choice([-1, 1], size=(dim, num))

        return vectors

    @staticmethod
    def compute_sim(vsa, vectors_1, vectors_2):
        # first transpose the arrays
        vectors_1 = vectors_1.T
        vectors_2 = vectors_2.T
        sim_matrix = np.zeros((vectors_1.shape[0], vectors_2.shape[0]))
        np.warnings.filterwarnings('ignore')
        if vsa in ['MAP_B', 'MAP_C', 'HRR', 'HRR_VTB', 'NONE', 'MAP_I', 'MBAT', 'Proj', 'FHRR_cos']:
            # cosine similarity
            sim_matrix = (vectors_1 / np.sqrt(np.sum(vectors_1**2, axis=1, keepdims=True))) @ (vectors_2 / np.sqrt(np.sum(vectors_2**2, axis=1, keepdims=True))).T
        elif vsa == 'BSC':
            # hamming distance
            sim_matrix = 1 - np.array([[hamming(vec1, vec2) for vec2 in vectors_2] for vec1 in vectors_1])
        elif vsa in ['BSDC', 'BSDC_SHIFT', 'BSDC_25', 'BSDC_SEG', 'BSDC_THIN']:
            # overlap
            vectors_1 = vectors_1 > 0
            vectors_2 = vectors_2 > 0
            sim_matrix = np.dot(vectors_1,vectors_2.T)
        elif vsa in ['FHRR_fft', 'FHRR']:
            # average of cosine of distance
            # convert to complex values
            sim_matrix = np.real(np.exp(vectors_1*1j) @ np.exp(vectors_2*1j).T) / vectors_1.shape[1]
        else:
            print('Representation is not defined!')
        np.warnings.filterwarnings('default')
        return sim_matrix

    @staticmethod
    def bind_vectors(vsa, vectors_1, vectors_2, density):
        default_density = 1/np.sqrt(vectors_1.shape[0])

        bound_vectors = None

        if vsa in ['MAP_B', 'MAP_C', 'MAP_I']:
            bound_vectors = vectors_1 * vectors_2
        elif vsa == 'BSC':
            bound_vectors = np.double(np.add(vectors_1, vectors_2) == 1)
        elif vsa == 'BSDC':
            values_disj = np.double(np.add(vectors_1, vectors_2))
            bound_vectors = cdt(values_disj, 50, 1/np.sqrt(vectors_1.shape[0]))
        elif vsa == 'BSDC_SHIFT':
            idx = np.arange(1, vectors_1.shape[0] + 1) @ vectors_1
            # shift each column with a specific index number
            bound_vectors = np.zeros_like(vectors_1)
            for i in range(vectors_1.shape[1]):
                bound_vectors[:, i] = np.roll(vectors_2[:, i], int(idx[i]))
        elif vsa == 'HRR':
            ccirc = np.fft.ifft(np.fft.fft(vectors_1, vectors_1.shape[0], axis=0) * np.fft.fft(vectors_2, vectors_2.shape[0], axis=0), axis=0)
            bound_vectors = ccirc
        elif vsa == 'FHRR':
            bound_vectors = np.mod(vectors_1 + vectors_2 + np.pi, 2 * np.pi) - np.pi
        elif vsa == 'BSDC_SEG':

            # Sparse vectors with segments
            dim = vectors_1.shape[0]
            num_segments = int(np.floor(dim * density))
            size_segments = int(np.floor(dim / num_segments))
            num_vecs = vectors_1.shape[1]

            role = vectors_1[:num_segments * size_segments, :]
            filler = vectors_2[:num_segments * size_segments, :]

            # First part of the vector
            role_segments = role.reshape((size_segments, num_segments, num_vecs), order='F')
            filler_segments = filler.reshape((size_segments, num_segments, num_vecs), order='F')

            role_idx = np.flatnonzero(role_segments)
            filler_idx = np.flatnonzero(filler_segments)

            role_rows, _, _ = np.unravel_index(role_idx, role_segments.shape)
            filler_rows, filler_cols, filler_tables = np.unravel_index(filler_idx, filler_segments.shape)

            result_rows = np.mod(role_rows[filler_cols * filler_tables] + filler_rows - 1, size_segments) + 1

            bound_vectors = np.zeros_like(role_segments)
            idx = np.unravel_index(result_rows, role_segments.shape)
            bound_vectors[idx] = 1
            bound_vectors = bound_vectors.reshape((size_segments * num_segments, num_vecs), order='F')

            bound_vectors_part2 = np.zeros_like(bound_vectors)
            if num_segments * size_segments != dim:
                filler = vectors_2[num_segments * size_segments:, :]
                bound_vectors_part2[:filler.shape[0], :] = filler

            bound_vectors = np.concatenate([bound_vectors, bound_vectors_part2], axis=0)
        return bound_vectors

    @staticmethod
    def unbind_vectors(vsa, vectors_1, vectors_2, density):
        if vsa in ['MAP_B', 'MAP_C', 'MAP_I']:
        # elementwise multiplication
            unbound_vectors = vectors_1 * vectors_2
        elif vsa == 'BSC':
            # efficient xor implementation for multi inputs
            unbound_vectors = np.double(np.add(vectors_1, vectors_2) == 1)
        elif vsa == 'BSDC':
            # find the most similar item in item_mem
            print('There is no specific unbind operator for the selected VSA - use the finding of the most similar vectors in item memory instead!')
        elif vsa == 'BSDC_SHIFT':
            # calculate the shift number (sum of all ones-index)
            idx = np.arange(1, vectors_1.shape[0]+1).dot(vectors_1)
            # shift each column with specific index number
            unbound_vectors = np.zeros(vectors_1.shape)
            for i in range(idx.size):
                unbound_vectors[:, i] = np.roll(vectors_2[:, i], -int(idx[i]))
        elif vsa == 'HRR':
            # involution as approximate inverse
            inverse = np.vstack((vectors_1[0, :], np.flip(vectors_1[1:, :], axis=0)))
            unbound_vectors = np.fft.ifft(np.fft.fft(inverse, vectors_1.shape[0], axis=0) * np.fft.fft(vectors_2, vectors_2.shape[0], axis=0), axis=0)
        elif vsa == 'FHRR':
            # complex multiplication with negative 'role' vector
            unbound_vectors = np.arctan2(np.sin(vectors_2 - vectors_1), np.cos(vectors_2 - vectors_1))
        elif vsa == 'BSDC_SEG':
            # Sparse vectors with segments
            dim = vectors_1.shape[0]
            num_segments = int(np.floor(dim * density))
            num_vecs = vectors_1.shape[1]
            size_segments = int(np.floor(dim / num_segments))
            role = vectors_1[:num_segments * size_segments, :]
            filler = vectors_2[:num_segments * size_segments, :]

            # First part of the vector
            role_segments = np.reshape(role, (size_segments, num_segments, num_vecs))
            filler_segments = np.reshape(filler, (size_segments, num_segments, num_vecs))

            # Initialize unbound_vectors with zeros
            unbound_vectors = np.zeros(role_segments.shape)

            # Loop through indices
            for filler_table in range(num_segments):
                for filler_col in range(num_vecs):
                    for role_row in range(size_segments):
                        # Calculate result_rows for each index
                        result_rows = np.mod(filler_segments[:, filler_table, filler_col] - role_row - 1, size_segments) + 1
                        result_rows = result_rows.astype(int)
                        # Convert indices to flat indices
                        idx = np.ravel_multi_index((result_rows, filler_col, filler_table), role_segments.shape)

                        # Update unbound_vectors at the calculated indices
                        unbound_vectors.flat[idx] = 1

            # Reshape unbound_vectors
            unbound_vectors = np.reshape(unbound_vectors, (size_segments * num_segments, num_vecs))

            # If there is a remaining part
            unbound_vectors_part2 = np.empty((0, num_vecs))
            if num_segments * size_segments != dim:
                filler = vectors_2[num_segments * size_segments:, :]
                unbound_vectors_part2 = filler

            # Stack unbound_vectors and unbound_vectors_part2 vertically
            unbound_vectors = np.vstack((unbound_vectors, unbound_vectors_part2))
        return unbound_vectors

    @staticmethod
    def bundle_vectors(vsa, vectors_1, vectors_2, normalize=None, density=None, max_density=None):
        if normalize is None:
            normalize = False

        if density is None:
            density = 0.5

        if max_density is None:
            max_density = 1

        if vsa == 'MAP_C':
            bundled_vectors = vectors_1 + vectors_2
        elif vsa == 'MAP_B':
            bundled_vectors = vectors_1 + vectors_2
        elif vsa == 'MAP_I':
            bundled_vectors = vectors_1 + vectors_2
        elif vsa == 'BSC':
            bundled_vectors = vectors_1 * vectors_2
        elif vsa == 'BSDC':
            bundled_vectors = vectors_1 * vectors_2
        elif vsa == 'BSDC_SHIFT':
            bundled_vectors = vectors_1 * vectors_2
        elif vsa == 'HRR':
            bundled_vectors = vectors_1 + vectors_2
        elif vsa == 'HRR_VTB':
            bundled_vectors = vectors_1 + vectors_2
        elif vsa == 'FHRR':
            bundled_vectors = vectors_1 + vectors_2
        elif vsa == 'BSDC_SEG':
            bundled_vectors = vectors_1 * vectors_2
        elif vsa == 'MBAT':
            bundled_vectors = vectors_1 + vectors_2
        else:
            bundled_vectors = vectors_1 + vectors_2

        if normalize:
            bundled_vectors = bundled_vectors / np.sqrt(np.sum(bundled_vectors**2, axis=0))

        if density < max_density:
            bundled_vectors = bundled_vectors * (density / max_density)

        return bundled_vectors

    @staticmethod
    def fractional_binding(vsa, vector, k):
        if vsa == 'MAP_C':
            v = vector
        elif vsa == 'MAP_B':
            v = vector
        elif vsa == 'MAP_I':
            v = vector
        elif vsa == 'BSC':
            v = vector
        elif vsa == 'BSDC':
            v = vector
        elif vsa == 'BSDC_SHIFT':
            v = vector
        elif vsa == 'HRR':
            v = vector
        elif vsa == 'HRR_VTB':
            v = vector
        elif vsa == 'FHRR':
            v = vector
        elif vsa == 'BSDC_SEG':
            v = vector
        elif vsa == 'MBAT':
            v = vector
        else:
            v = vector

        return v

    @staticmethod
    def convert_vectors(vsa, vector_array, density):
        if vsa == 'MAP_C':
            converted_vectors = vector_array
        elif vsa == 'MAP_B':
            converted_vectors = vector_array
        elif vsa == 'MAP_I':
            converted_vectors = vector_array
        elif vsa == 'BSC':
            converted_vectors = vector_array
        elif vsa == 'BSDC':
            converted_vectors = vector_array
        elif vsa == 'BSDC_SHIFT':
            converted_vectors = vector_array
        elif vsa == 'HRR':
            converted_vectors = vector_array
        elif vsa == 'HRR_VTB':
            converted_vectors = vector_array
        elif vsa == 'FHRR':
            converted_vectors = vector_array
        elif vsa == 'BSDC_SEG':
            converted_vectors = vector_array
        elif vsa == 'MBAT':
            converted_vectors = vector_array
        else:
            converted_vectors = vector_array

        return converted_vectors

**demo_cpu:**

Demonstrates the usage of the Vector Symbolic Architecture (VSA) in Python for various VSA types. It loads images from a directory, converts them to vectors, and performs VSA operations like bundling, binding/unbinding, and vector retrieval. The code iterates through different VSA types, including MAP, BSC, BSDC, HRR, FHRR, and more. It measures the elapsed time for each operation, prints similarity values where applicable, and demonstrates finding vectors using the item memory and handling noise vectors.

In [20]:
import os
import sys
import numpy as np
from PIL import Image
import time

# Suppress warnings
sys.stderr = open(os.devnull, 'w')
sys.stderr = sys.__stderr__

# create the object of a specific VSA type
vsa_types = ['MAP_C', 'MAP_B', 'MAP_I', 'BSC', 'BSDC', 'BSDC_SHIFT', 'HRR', 'FHRR']
for type_idx in range(len(vsa_types)):
    type = vsa_types[type_idx]
    VSA = vsa_env(type, 1024)

    # Load images from a directory in Google Drive
    imageDir = '/content/drive/MyDrive/GardensPointWalking/night_right'
    imageFiles = [file for file in os.listdir(imageDir) if file.endswith('.jpg')]


    # Initialize the combined vector
    combinedVector = []

    # Iterate through each image and convert it to a vector
    for i in range(len(imageFiles)//2):
        currentFileName = imageFiles[i]
        currentImagePath = os.path.join(imageDir, currentFileName)

        # Read the image
        img = Image.open(currentImagePath)
        squareSize = 256
        img = img.resize((squareSize, squareSize)).convert('L')
        imgVector = np.array(img).reshape(-1, 1)
        combinedVector.append(imgVector)
    combinedVector = np.squeeze(np.array(combinedVector))
    combinedVector = np.reshape(combinedVector, (combinedVector.shape[1], combinedVector.shape[0]))
    VSA.add_vector(combinedVector)
    combinedVector1 = []

    # Iterate through each image and convert it to a vector
    for i in range(len(imageFiles)//2, len(imageFiles)):
        currentFileName = imageFiles[i]
        currentImagePath = os.path.join(imageDir, currentFileName)

        # Read the image
        img = Image.open(currentImagePath)
        squareSize = 256
        img = img.resize((squareSize, squareSize)).convert('L')
        imgVector = np.array(img).reshape(-1, 1)
        combinedVector1.append(imgVector)
    combinedVector1 = np.squeeze(np.array(combinedVector1))
    combinedVector1 = np.reshape(combinedVector1, (combinedVector1.shape[1], combinedVector1.shape[0]))
    rows, cols = combinedVector.shape
    rows1, cols1 = combinedVector1.shape

    if cols != cols1:
        # Remove the last index of the longer vector to make them equal
        minLength = min(cols, cols1)
        combinedVector1 = combinedVector1[:, :minLength]

    ## 1. bundling

    print('Bundling vectors for VSA type: ' + type)
    start_time = time.time()
    if type != 'FHRR':
        bundle = VSA.bundle(combinedVector, combinedVector1)  # Use the vectors directly
    end_time = time.time()
    print(f'Elapsed time: {end_time-start_time:.5f} seconds')

    ## 2. binding / unbinding

    if type == 'BSDC':
        print('Binding vectors for VSA type: ' + type)
        start_time = time.time()
        v_array_1 = VSA.add_vector(combinedVector,add_item=1)
        v_array_2 = VSA.add_vector(combinedVector1,add_item=1)
        bound_v = VSA.bind(v_array_1.astype(float), v_array_2.astype(float))  # Use the vectors directly
        end_time = time.time()
        print(f'Elapsed time: {end_time-start_time:.5f} seconds')
        start_time = time.time()

        # Use similarity search for BSDC
        v_clean, name, s = VSA.find_k_nearest(np.array(bound_v), 1)
        sim_v1 = s
    else:
        print('Binding vectors for VSA type: ' + type)
        start_time = time.time()
        v_array_1 = VSA.add_vector(combinedVector)
        v_array_2 = VSA.add_vector(combinedVector1)
        bound_v = VSA.bind(v_array_1.astype(float), v_array_2.astype(float))  # Use the vectors directly
        end_time = time.time()
        print(f'Elapsed time: {end_time-start_time:.5f} seconds')
        start_time = time.time()

        # Regular unbinding operation for other VSA types
        r = VSA.unbind(v_array_1.astype(float), bound_v.astype(float))
        sim_v1 = VSA.sim(r.astype(float), v_array_1.astype(float))
        sim_v2 = VSA.sim(r.astype(float), v_array_2.astype(float))
    end_time = time.time()
    print('------- unbinding:')
    print(f'Elapsed time: {end_time-start_time:.5f} seconds')

    ## 3. use the item memory to find vectors

    print('Finding k vectors for VSA type: ' + type)

    # fill the item memory with random vectors
    VSA = vsa_env(type, 65536)
    v_array_1 = VSA.add_vector(combinedVector[:,1][:, np.newaxis].astype(float))
    v = VSA.add_vector(add_item=1)
    v_clean, name, s = VSA.find_k_nearest(v.astype(float), 1)

    # Convert similarity to percentage for BSDC types
    if type == 'BSDC' or type == 'BSDC_SHIFT' or type == 'BSDC_SEG':
        # Extract the first element from the array if s is an array
        similarity_value = s[0][0] if isinstance(s, np.ndarray) else s
        # Convert raw similarity scores to decimal format for BSDC types
        similarity_decimal = similarity_value / 1024
        print(f'Found probe vector with similarity of {similarity_decimal:.5f}')
    else:
        # Extract the first element from the array if s is an array
        similarity_value = s[0][0] if isinstance(s, np.ndarray) else s
        print(f'Found probe vector with similarity of {similarity_value:.5f}')

    # bundle the probe with noise vector
    noise = VSA.add_vector()
    bundle = VSA.bundle(v, noise)
    v_clean, name, s = VSA.find_k_nearest(bundle, 1)

    # Convert similarity to decimal format for BSDC types
    if type == 'BSDC' or type == 'BSDC_SHIFT' or type == 'BSDC_SEG':
        # Extract the first element from the array if s is an array
        similarity_value = s[0][0] if isinstance(s, np.ndarray) else s
        similarity_decimal = similarity_value / 1024
        print(f'Found noisy vector probe with similarity of {similarity_decimal:.5f}')
    else:
        # Extract the first element from the array if s is an array
        similarity_value = s[0][0] if isinstance(s, np.ndarray) else s
        print(f'Found noisy vector with similarity of {similarity_value:.5f}')

Bundling vectors for VSA type: MAP_C
Elapsed time: 0.02827 seconds
Binding vectors for VSA type: MAP_C
Elapsed time: 0.07503 seconds
------- unbinding:
Elapsed time: 0.44154 seconds
Finding k vectors for VSA type: MAP_C
Found probe vector with similarity of 1.00000
Found noisy vector with similarity of 0.70669
Bundling vectors for VSA type: MAP_B
Elapsed time: 0.02131 seconds
Binding vectors for VSA type: MAP_B
Elapsed time: 0.05724 seconds
------- unbinding:
Elapsed time: 0.44021 seconds
Finding k vectors for VSA type: MAP_B
Found probe vector with similarity of 1.00000
Found noisy vector with similarity of 0.86581
Bundling vectors for VSA type: MAP_I
Elapsed time: 0.02170 seconds
Binding vectors for VSA type: MAP_I
Elapsed time: 0.06126 seconds
------- unbinding:
Elapsed time: 0.41099 seconds
Finding k vectors for VSA type: MAP_I
Found probe vector with similarity of 1.00000
Found noisy vector with similarity of 1.00000
Bundling vectors for VSA type: BSC
Elapsed time: 0.02024 seconds